### A/B Testing Quick Visualization by Ido Haber
- V1.0 
- 02_07_2024

In [1]:
'''
Import all critical libraries. pip3 install if do not ahve PyPDF2/PIL.

'''
from PIL import Image, ImageDraw, ImageFont
import os
import time
import PyPDF2
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.colors import black, white
from reportlab.lib.units import inch

## Creates base PDF with columns for images (Version 1)
1. change path of directories and number of directories included in function.
2. Keep in mind target size based on number of directories. 

- Pair images from multiple directories based on their **exact** names and save them as a PDF.
- If a special string is provided, images containing that string in their name are placed **individually** a slide.

In [ ]:
def pair_imgs_to_pdf(dirs, output_pdf, target_size=None, special_string=None):
    """
    :param dirs: List of directories containing PNG, JPG, JPEG files.
    :param output_pdf: Output PDF file name.
    :param target_size: Optional tuple (width, height) to resize each individual image.
    :param special_string: Optional string to identify special images for individual placement. If None, no special treatment.
    """
    start_time = time.time()  # Start timing
    all_images = []

    # Collect images from each directory
    image_files = {dir: sorted([f for f in os.listdir(dir) if f.endswith(('.png', '.jpg', '.jpeg'))]) for dir in dirs}

    # Find matching images across directories
    paired_images = set()
    for dir, files in image_files.items():
        for file in files:
            if special_string and special_string in file:
                # Process special images
                img_path = os.path.join(dir, file)
                img = Image.open(img_path).convert('RGB')
                if target_size:
                    img = img.resize(target_size, Image.Resampling.LANCZOS)
                all_images.append(img)
            elif file not in paired_images:
                # Pair matching images
                imgs = []
                for other_dir in dirs:
                    other_path = os.path.join(other_dir, file)
                    if os.path.exists(other_path):
                        img = Image.open(other_path).convert('RGB')
                        if target_size:
                            img = img.resize(target_size, Image.Resampling.LANCZOS)
                        imgs.append(img)
                        paired_images.add(file)

                if len(imgs) > 1:
                    total_width = sum(img.width for img in imgs)
                    max_height = max(img.height for img in imgs)
                    new_img = Image.new('RGB', (total_width, max_height))

                    x_offset = 0
                    for img in imgs:
                        new_img.paste(img, (x_offset, 0))
                        x_offset += img.width

                    all_images.append(new_img)

    # Check if there are images to be saved in the PDF
    if not all_images:
        print("No images to create PDF.")
        return

    # Save all images to a PDF
    all_images[0].save(output_pdf, save_all=True, append_images=all_images[1:])
    print(f"PDF created successfully: {output_pdf}")

    end_time = time.time()  # End timing
    print(f"The function took {end_time - start_time} seconds to run.")

dir1 = '/path1'
dir2 = '/path2'

# Add or remove directories as needed
# Replace None based on special string 
pair_imgs_to_pdf([dir1, dir2], 'columns_images.pdf', target_size=(1000, 1000), special_string=None) 


## Creates base PDF with columns for images (Version 2)
1. change path of directories and number of directories included in function.
2. Keep in mind target size based on number of directories. 

- Pairs images from multiple directories based on the order of the files in the directory (not based on name)

In [ ]:
def pair_imgs_to_pdf(dirs, output_pdf, target_size=None):
    """
    :param dirs: List of directories containing PNG, JPG, JPEG files.
    :param output_pdf: Output PDF file name.
    :param page_titles: List of titles for each PDF page.
    :param target_size: Optional tuple (width, height) to resize each individual image.
    """
    start_time = time.time()  # Start timing
    # Initialize list for the paired images and file names from each directory
    paired_images = []
    all_files = []

    # Process each directory
    for dir in dirs:
        files = [f for f in os.listdir(dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
        files.sort()
        all_files.append(files)

    # Check if any directory is empty 
    if any(not files for files in all_files):
        print("PNG or JPEG files missing in one or more directories.")
        return
         
    # Process and pair the images
    for i, files in enumerate(zip(*all_files)):
        imgs = []
        for dir, file in zip(dirs, files):
            img_path = os.path.join(dir, file)
            img = Image.open(img_path)

            if target_size:
                img = img.resize(target_size, Image.Resampling.LANCZOS)

            img = img.convert('RGB')
            imgs.append(img)

        # Calculate dimensions for the new image
        total_width = sum(img.width for img in imgs)
        max_height = max(img.height for img in imgs)

        # Create a new image and paste the individual images
        new_img = Image.new('RGB', (total_width, max_height))
        x_offset = 0
        for img in imgs:
            new_img.paste(img, (x_offset, 0))
            x_offset += img.width
        
        paired_images.append(new_img)

    # Check if there are images to be saved in the PDF
    if not paired_images:
        print("No images to create PDF.")
        return

    # Save all paired images to a PDF
    paired_images[0].save(output_pdf, save_all=True, append_images=paired_images[1:])
    print(f"PDF created successfully: {output_pdf}")

    end_time = time.time()  # End timing
    print(f"The function took {end_time - start_time} seconds to run.")
    
dir1 = '/path1' # Replace with the path to your first directory
dir2 = '/path2' 
dir3 = '/path3' 
dir4 = '/path4' 


"""
add or remove number of dirs from list based on your needs
set your desired target size per image
set name for pdf
"""

pair_imgs_to_pdf([dir1,dir2], 'base_columns_v2.pdf', target_size=(1000,1000)) 

## Creates base PDF with rows of images

In [4]:

def stack_pngs_in_pdf(dir_path, output_pdf, images_per_page, target_size=None):
    """
    :param dir_path: Directory where images images are stored.
    :param output_pdf: Output PDF file name.
    :param images_per_page: Number of images to stack on each page of the PDF.
    :param target_size: Optional tuple (width, height) to resize each individual image.
    """
    start_time = time.time()  # Start timing

    # Get all PNG files in the directory
    files = [f for f in os.listdir(dir_path) if f.endswith('.png')]
    files.sort()  # Sort files to maintain order

    if not files:
        print("No PNG files found in the directory.")
        return

    # Initialize list for the stacked images
    stacked_images_pages = []

    for i in range(0, len(files), images_per_page):
        imgs = [Image.open(os.path.join(dir_path, file)) for file in files[i:i+images_per_page]]
        
        if target_size:
            imgs = [img.resize(target_size, Image.Resampling.LANCZOS) for img in imgs]

        # Convert images to RGB
        imgs = [img.convert('RGB') for img in imgs]

        # Calculate dimensions for the new stacked image
        max_width = max(img.width for img in imgs)
        total_height = sum(img.height for img in imgs)

        # Create a new image for the stacked images
        new_img = Image.new('RGB', (max_width, total_height))
        y_offset = 0
        for img in imgs:
            new_img.paste(img, (0, y_offset))
            y_offset += img.height
        
        stacked_images_pages.append(new_img)

    # Check if there are images to be saved in the PDF
    if not stacked_images_pages:
        print("No images to create PDF.")
        return

    # Save all stacked images to a PDF
    stacked_images_pages[0].save(output_pdf, save_all=True, append_images=stacked_images_pages[1:])
    print(f"PDF created successfully: {output_pdf}")

    end_time = time.time()  # End timing
    print(f"The function took {end_time - start_time} seconds to run.")

# Update parameters:
images_per_page = 3
dir_path = '/path' 
stack_pngs_in_pdf(dir_path, 'stacked_images.pdf', images_per_page)



PDF created successfully: stacked_images.pdf
The function took 1.201408863067627 seconds to run.


## Create PDF2 with titles for figures
This block creates the background slide where you could add titles and subtitles to accompany your figures.

- Make sure to change X dimention with respect to the number of figures in PDF1. 2Figs=2000, 3Figs=3000.


In [ ]:
def create_titles_pdf(titles, output_pdf, target_size=None):

    c = canvas.Canvas(output_pdf, pagesize=target_size)
    width, height = target_size
    font_size = 30

   # Define the consistent subtitle. The words will be separated equally along the x axis of the page.
    bottom_title = "subtitle_1 subtitle_2"
    bottom_font_size = 20  
    
   # Split the bottom title into words
    bottom_title_words = bottom_title.split()

    #Calculate the total width of all words and the spacing needed
    total_words_width = sum(c.stringWidth(word, "Helvetica-Bold", bottom_font_size) for word in bottom_title_words)
    total_spacing = width - total_words_width
    space_between_words = total_spacing / (len(bottom_title_words) + 1)
    
    for title in titles:
        # Set up the title font
        c.setFont("Helvetica-Bold", font_size)
        
        # Calculate text width and height
        text_width = c.stringWidth(title, "Helvetica-Bold", font_size)
        text_height = font_size

        # Calculate background and text positions
        background_width = text_width + 2000
        background_height = text_height + 50
        background_x = (width - background_width) // 2
        background_y = height - background_height - 25 # Adjust position as needed

        # Draw the background
        c.setFillColor(white)
        c.rect(background_x, background_y, background_width, background_height, stroke=0, fill=1)

        # Draw the title text
        text_x = (width - text_width) / 2
        text_y = background_y + (background_height - text_height) / 2
        c.setFillColor(black)
        c.drawString(text_x, text_y, title)

        # Draw the consistent bottom title at specific x-coordinates
        bottom_text_y = 30  # Y-coordinate for bottom title
        x_positions = [120, 1120, 2120]  # X-coordinates for each word

        # Draw the bottom title
        c.setFont("Helvetica-Bold", bottom_font_size)
        x_position = space_between_words
        for word in bottom_title_words:
            c.drawString(x_position, 30, word)  # Y-coordinate adjusted for bottom title
            word_width = c.stringWidth(word, "Helvetica-Bold", bottom_font_size)
            x_position += word_width + space_between_words

        # Move to the next page
        c.showPage()

    c.save()

page_titles = ['Example title #1',
               'Example title #2',
               'Example title #4',
               'Example title #5',
               'Example title #6'
]

create_titles_pdf(page_titles, "titles.pdf",  target_size=(2000,1100))

## Creates Final PDF 

- takes the first PDF with figures and places it on top of the second PDF containing titles. 
- no setting up required. Just make sure to names of the files in question match the function.

In [ ]:
def merge_pdfs(base_pdf_path, overlay_pdf_path, output_pdf_path):
    # Open the base PDF
    base_pdf = PyPDF2.PdfReader(open(base_pdf_path, "rb"))

    # Open the overlay PDF
    overlay_pdf = PyPDF2.PdfReader(open(overlay_pdf_path, "rb"))

    # Create a PDF writer
    pdf_writer = PyPDF2.PdfWriter()

    # Check and adjust dimensions if necessary
    base_page = base_pdf.pages[0]
    overlay_page = overlay_pdf.pages[0]
    base_height = base_page.mediabox[3]
    overlay_height = overlay_page.mediabox[3]
    
    # Adjust base page size if overlay is taller
    if overlay_height > base_height:
        for page in base_pdf.pages:
            page.mediabox[3] = overlay_height
    
   # Iterate through base PDF pages and merge
    for page_num in range(len(base_pdf.pages)):
        base_page = base_pdf.pages[page_num]
        # Check if there's a corresponding overlay page, else use the last overlay page
        overlay_page = overlay_pdf.pages[min(page_num, len(overlay_pdf.pages) - 1)]

        # Merge the pages
        base_page.merge_page(overlay_page)

        # Add the merged page to the writer
        pdf_writer.add_page(base_page)

    # Write the merged PDF to file
    with open(output_pdf_path, "wb") as out:
        pdf_writer.write(out)
        
        
# Comment in/out base on need.
merge_pdfs("columns_images.pdf","titles_columns.pdf", "columns_final.pdf")
merge_pdfs("stacked_images.pdf","title_stacked.pdf", "stacked_final.pdf")